<a href="https://colab.research.google.com/github/Redikins-JH/DeepLearning-Test/blob/master/busan_stock_price_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 웹 크롤링 경험해보기

### Q4. get_url 함수를 정의하시오
* 상장사의 '상장 코드' 를 이용하여 (예. 삼성전자:005930, LG전자:066570)
* naver 금융 웹페이지의 주식 시세 정보를 검색하는 url을 작성하시오
* naver 주식 시세 정보 검색 url : https://finance.naver.com/item/sise_day.nhn?code=`상장 코드`
* 작성한 함수를 이용하여 삼성전자 주식 조회를 위한 naver 주식 시세 정보 검색 url 정보를 출력하시오

In [1]:
import pandas as pd

def get_url():
  # 삼성전자의 code 값은 005930 임을 알고 크롤링한다고 가정
  url = 'https://finance.naver.com/item/sise_day.nhn?code={code}'.format(code = '005930')
  print("request URL = {}".format(url))
  return url

In [2]:
item_name='삼성전자' # code = 005930
url = get_url()

request URL = https://finance.naver.com/item/sise_day.nhn?code=005930


### url을 이용한 주식시세 정보 호출 및 html 파싱을 위한 bs4 라이브러리 임포트

In [3]:
import requests
from bs4 import BeautifulSoup

### Q5. Q4에서 생성한 naver 주식 시세정보 조회 url을 이용하여 200일 간의 삼성전자의 주식 시세 정보를 리스트 객체에 저장하시오
* list 의 shape = (200(일수), 7(날짜, 종가, 차액, 시가, 상한, 하한, 거래량))

In [4]:
df = pd.DataFrame()

priceList = []

for page in range(1, 21):
  pg_url = '{url}&page={page}'.format(url=url, page=page)

  
  response= requests.get(pg_url)
  soup = BeautifulSoup(response.content, 'lxml')

  trList = soup.body.find_all(name='tr', attrs={'onmouseover':'mouseOver(this)'})
  
  for tr in trList:
    priceList.append([td.span.string.strip() for td in tr.find_all(name='td')])

### Q6. priceList 정보를 이용하여 df 이름을 dataframe 객체 생성
* 컬럼의 이름은 'date', 'close', 'diff', 'open', 'high', 'low', 'volume' 롤 지정

In [5]:
df = pd.DataFrame(priceList,
                  columns=['date', 'close', 'diff', 'open', 'high', 'low', 'volume']
                  )
df.head()

,date,close,diff,open,high,low,volume
0,2020.07.30,"59,200",200,"59,700","60,100","59,100","11,874,607"
1,2020.07.29,"59,000",400,"60,300","60,400","58,600","36,476,611"
2,2020.07.28,"58,600","3,000","57,000","58,800","56,400","48,431,566"
3,2020.07.27,"55,600","1,400","54,300","55,700","54,300","21,054,421"
4,2020.07.24,"54,200",100,"54,000","54,400","53,700","10,994,535"


### Q7. df 객체에서 Nan 제거

In [6]:
df = df.dropna()
df.head()

,date,close,diff,open,high,low,volume
0,2020.07.30,"59,200",200,"59,700","60,100","59,100","11,874,607"
1,2020.07.29,"59,000",400,"60,300","60,400","58,600","36,476,611"
2,2020.07.28,"58,600","3,000","57,000","58,800","56,400","48,431,566"
3,2020.07.27,"55,600","1,400","54,300","55,700","54,300","21,054,421"
4,2020.07.24,"54,200",100,"54,000","54,400","53,700","10,994,535"


### df 객체에 각 원소들의 값이 object로 되어 있기 때문에 분석을 위해 float type으로 

In [7]:
df.dtypes

date      object
close     object
diff      object
open      object
high      object
low       object
volume    object
dtype: object

In [16]:
import locale
from locale import atof

locale.setlocale(locale.LC_NUMERIC, '')

df['date'] = pd.to_datetime(df['date'])


df[['close', 'diff', 'open', 'high', 'low', 'volume']] = \
df[['close', 'diff', 'open', 'high', 'low', 'volume']].applymap(atof)
df = df.sort_values(by=['date'], ascending=True)

AttributeError: ignored

In [10]:
df.dtypes

date      datetime64[ns]
close            float64
diff             float64
open             float64
high             float64
low              float64
volume           float64
dtype: object

In [11]:
df.head()

,date,close,diff,open,high,low,volume
199,2019-10-14,50000.0,850.0,50000.0,50300.0,49850.0,11130635.0
198,2019-10-15,50100.0,100.0,49900.0,50200.0,49900.0,6058105.0
197,2019-10-16,50700.0,600.0,50700.0,50900.0,50400.0,9136282.0
196,2019-10-17,50500.0,200.0,50500.0,50600.0,50100.0,6701380.0
195,2019-10-18,49900.0,600.0,50300.0,50900.0,49650.0,8483106.0


### 삼성 전자 종가 기준 일별 주가 시계열 차트 조회

In [12]:
# 시계열 차트 생성을 위한 라이브러리 임포트
import plotly.graph_objects as go

In [13]:
# 차트 생성을 위한 데이터 생성
trace = go.Scatter(x=df['date'],
                   y=df['close'],
                   name=item_name)

data = [trace]

In [14]:
# 시계열 차트의 레이웃 정보 생성
layout = dict(
    title='{}의 종가(close) Time Series'.format(item_name),
    # x축 데이터 타입 지정(date)
    xaxis = dict(
        rangeslider=dict(),
        type='date'
        ),
)

In [15]:
# 차트 객체 생성 및 조회
fig = go.Figure(data=data, layout=layout)
fig.show()